In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.chdir('/content/drive/MyDrive/project')
!ls

Data				 Plots	  Sentiment_Analysis.ipynb
Model_Prediction_Pipeline.ipynb  Results  Sentiment.ipynb
Models				 Scalers  Stacked_LSTM_code.ipynb


# Single day prediction

In [4]:
import yfinance as yf
import numpy as np
import pickle
import joblib
import json
from pandas.tseries.offsets import BDay
from tensorflow.keras.models import load_model

In [ ]:
class TestModel:

  def __init__(self, ticker: str):
    self.ticker = ticker
    self.model = None
    self.hyperparams = None
    self.scaled_features, self.scaled_target = None, None
    self.feature_scaler, self.target_scaler = None, None
    self.dataset = None

  # 1 ----> loads Scaler + Model + Hyperparams
  def load_all(self, ticker: str = None):
    # Load the model
    base_ticker = self.ticker.split(".")[0]
    path = f'Models/{base_ticker}_LSTM.keras'
    self.model = load_model(path)
    print(f'✅Loaded the model for {self.ticker}')

    # Load the Hyperparameters
    with open('Results/results.json', 'r') as f:
      best_params = json.load(f)
    self.hyperparams = best_params[self.ticker]['Hyperparameters']
    print(f'✅Loaded the hyperparameter for {self.ticker}')

    # Load the scalers
    with open(f'Scalers/{base_ticker}_feature_scaler.pkl', 'rb') as f:
      self.feature_scaler = joblib.load(f)

    with open(f'Scalers/{base_ticker}_target_scaler.pkl', 'rb') as f:
      self.target_scaler = joblib.load(f)

    print(f'✅Loaded the target & feature scaler for {self.ticker}')

  # 2 ----> Fetches + creates dataset
  def fetch_data(self):
    seq_len = self.hyperparams['sequence_length']
    self.dataset = yf.Ticker(self.ticker).history(period=f'{seq_len+10}d', interval='1d').dropna()
    print(f'✅Fetched the data for {self.ticker}')

    features = self.dataset[['Open', 'High', 'Low', 'Volume']]
    target = self.dataset[['Close']]

    self.scaled_features = self.feature_scaler.transform(features)
    self.scaled_target = self.target_scaler.transform(target)

  # 3 ----> Makes predictions
  def make_prediction(self):
    """Predict the next day's Close price using the saved LSTM model."""
    # Check prerequisites
    if self.model is None or self.scaled_features is None or self.target_scaler is None:
        raise ValueError("Model, scaled features, or target scaler not loaded. Ensure load_model() and fetch_data() are called.")

    seq_len = self.hyperparams['sequence_length']

    # Validate data size
    if len(self.scaled_features) < seq_len:
        raise ValueError(f"Insufficient data: need at least {seq_len} days, got {len(self.scaled_features)}.")

    # Prepare input: most recent seq_len days
    X_input = self.scaled_features[-seq_len:]  # Shape: (seq_len, 4)
    X_input = np.reshape(X_input, (1, seq_len, X_input.shape[1]))  # Shape: (1, seq_len, 4)

    # Predict next day's scaled Close
    pred_scaled = self.model.predict(X_input, verbose=0)
    predicted_close = self.target_scaler.inverse_transform(pred_scaled)[0][0]

    # Get dates
    last_date = self.dataset.index[-1]
    next_date = last_date + BDay(1)  # Next trading day (skips weekends/holidays)

    # Confidence based on recent volatility (last 5 days)
    recent_closes = self.dataset['Close'].tail(5).values
    if len(recent_closes) < 2:
        volatility = 0  # Fallback for insufficient data
    else:
        volatility = np.std(recent_closes) / np.mean(recent_closes)
    confidence = max(0, 1 - volatility * 10)
    confidence = round(min(confidence * 100, 99.9), 2)

    # Output results
    currency = "₹" if '.NS' in self.ticker else "$"
    print(f"\n📅 Last Available Date: {last_date.date()}")
    print(f"📅 Predicted Date: {next_date.date()} {'(Business Day)' if next_date.weekday() < 5 else '(Note: Weekend/Holiday)'})")
    print(f"💹 Predicted Next-Day Close for {self.ticker}: {currency}{predicted_close:.2f}")
    print(f"📈 Confidence Score: {confidence:.2f}% (based on recent volatility)")

  # 4 ----> Runs the pipeline
  def run_pipeline(self):
    self.load_all()
    self.fetch_data()
    self.make_prediction()

In [ ]:
obj = TestModel('TCS.NS')
obj.run_pipeline()

✅Loaded the model for TCS.NS
✅Loaded the hyperparameter for TCS.NS
✅Loaded the target & feature scaler for TCS.NS
✅Fetched the data for TCS.NS


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(



📅 Last Available Date: 2025-10-21
📅 Predicted Date: 2025-10-22 (Business Day))
💹 Predicted Next-Day Close for TCS.NS: ₹3064.58
📈 Confidence Score: 92.76% (based on recent volatility)


# Recursive multiday predictions'


In [29]:
import numpy as np
import yfinance as yf
import joblib, json
from tensorflow.keras.models import load_model
from pandas.tseries.offsets import BDay

class TestModel:
    def __init__(self, ticker: str):
        self.ticker = ticker
        self.model = None
        self.hyperparams = None
        self.scaled_features, self.scaled_target = None, None
        self.feature_scaler, self.target_scaler = None, None
        self.dataset = None
        self.future_predictions = []

    # 1 ----> Loads Scaler + Model + Hyperparams
    def load_all(self, ticker: str = None):
        base_ticker = self.ticker.split(".")[0]
        path = f'Models/{base_ticker}_LSTM.keras'
        self.model = load_model(path)
        print(f'✅ Loaded model for {self.ticker}')

        with open('Results/results.json', 'r') as f:
            best_params = json.load(f)
        self.hyperparams = best_params[self.ticker]['Hyperparameters']
        print(f'✅ Loaded hyperparameters for {self.ticker}')

        with open(f'Scalers/{base_ticker}_feature_scaler.pkl', 'rb') as f:
            self.feature_scaler = joblib.load(f)
        with open(f'Scalers/{base_ticker}_target_scaler.pkl', 'rb') as f:
            self.target_scaler = joblib.load(f)
        print(f'✅ Loaded feature & target scalers for {self.ticker}')

    # 2 ----> Fetches + Creates Dataset
    def fetch_data(self):
        seq_len = self.hyperparams['sequence_length']
        self.dataset = yf.Ticker(self.ticker).history(period=f'{seq_len+20}d', interval='1d').dropna()
        print(f'✅ Fetched latest data for {self.ticker}')

        features = self.dataset[['Open', 'High', 'Low', 'Volume']]
        target = self.dataset[['Close']]

        self.scaled_features = self.feature_scaler.transform(features)
        self.scaled_target = self.target_scaler.transform(target)

    # 3 ----> Predict Next-Day Close
    def make_prediction(self):
        if self.model is None or self.scaled_features is None or self.target_scaler is None:
            raise ValueError("Model, scaled features, or target scaler not loaded.")

        seq_len = self.hyperparams['sequence_length']
        if len(self.scaled_features) < seq_len:
            raise ValueError(f"Need at least {seq_len} days of data.")

        X_input = self.scaled_features[-seq_len:]
        X_input = np.reshape(X_input, (1, seq_len, X_input.shape[1]))

        pred_scaled = self.model.predict(X_input, verbose=0)
        predicted_close = self.target_scaler.inverse_transform(pred_scaled)[0][0]

        last_date = self.dataset.index[-1]
        next_date = last_date + BDay(1)

        recent_closes = self.dataset['Close'].tail(5).values
        volatility = np.std(recent_closes) / np.mean(recent_closes) if len(recent_closes) >= 2 else 0
        confidence = round(min(max(0, 1 - volatility * 10) * 100, 99.9), 2)

        currency = "₹" if '.NS' in self.ticker else "$"
        print(f"\n📅 Last Available Date: {last_date.date()}")
        print(f"📅 Predicted Date: {next_date.date()}")
        print(f"💹 Predicted Next-Day Close for {self.ticker}: {currency}{predicted_close:.2f}")
        print(f"📈 Confidence Score: {confidence:.2f}%")

        return predicted_close

    # 4 ----> Recursive Multi-Day Prediction
    def make_recursive_predictions(self, n_days=4):
        """Predicts the next n_days recursively using the trained LSTM model."""
        if self.model is None or self.scaled_features is None:
            raise ValueError("Model and scaled features must be loaded first.")

        seq_len = self.hyperparams['sequence_length']
        data = self.scaled_features.copy()  # to avoid modifying original data

        last_date = self.dataset.index[-1]
        self.future_predictions = []

        for i in range(n_days):
            X_input = data[-seq_len:]
            X_input = np.reshape(X_input, (1, seq_len, X_input.shape[1]))

            pred_scaled = self.model.predict(X_input, verbose=0)
            predicted_close = self.target_scaler.inverse_transform(pred_scaled)[0][0]

            next_date = last_date + BDay(i + 1)
            currency = "₹" if '.NS' in self.ticker else "$"
            self.future_predictions.append({
                "Date": next_date.date(),
                "Predicted_Close": float(predicted_close)
            })

            # ---- UPDATE INPUT SEQUENCE ----
            # Create a new row: use last known feature row but replace Close with predicted value
            last_row_features = self.dataset[['Open', 'High', 'Low', 'Volume']].iloc[-1].values
            new_feature_row = np.array(last_row_features).reshape(1, -1)
            new_feature_scaled = self.feature_scaler.transform(new_feature_row)
            data = np.vstack([data, new_feature_scaled])  # add new prediction

        # ---- PRINT RESULTS ----
        print("\n🔁 Recursive Multi-Day Forecast:")
        for pred in self.future_predictions:
            print(f"📅 {pred['Date']} → 💹 Predicted Close: {currency}{pred['Predicted_Close']:.2f}")

        return self.future_predictions

    # 4 ----> Return close prices
    def get_close_prices(self):
      Close_prices = []
      dates = []
      for dicti in self.future_predictions:
        Close_prices.append(dicti['Predicted_Close'])
        dates.append(dicti['Date'])
      return dates, Close_prices


    # 5 ----> Runs Full Pipeline
    def run_pipeline(self, recursive_days=0):
        self.load_all()
        self.fetch_data()

        if recursive_days > 0:
            return self.make_recursive_predictions(n_days=recursive_days)
        else:
            return self.make_prediction()

In [30]:
obj = TestModel('TCS.NS')
obj.run_pipeline(recursive_days=5) # use recursive days upto 5 days
dates, prices = obj.get_close_prices()

✅ Loaded model for TCS.NS
✅ Loaded hyperparameters for TCS.NS
✅ Loaded feature & target scalers for TCS.NS
✅ Fetched latest data for TCS.NS


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(



🔁 Recursive Multi-Day Forecast:
📅 2025-11-03 → 💹 Predicted Close: ₹3109.96
📅 2025-11-04 → 💹 Predicted Close: ₹3110.13
📅 2025-11-05 → 💹 Predicted Close: ₹3110.77
📅 2025-11-06 → 💹 Predicted Close: ₹3111.50
📅 2025-11-07 → 💹 Predicted Close: ₹3110.65


In [38]:
import numpy as np
import yfinance as yf
import joblib, json
from tensorflow.keras.models import load_model
from pandas.tseries.offsets import BDay

class TestModel2:
    def __init__(self, ticker: str):
        self.ticker = ticker
        self.model = None
        self.hyperparams = None
        self.scaled_features, self.scaled_target = None, None
        self.feature_scaler, self.target_scaler = None, None
        self.dataset = None
        self.future_predictions = []

    # 1 ----> Loads Scaler + Model + Hyperparams
    def load_all(self, ticker: str = None):
        base_ticker = self.ticker.split(".")[0]
        path = f'Models/{base_ticker}_LSTM.keras'
        self.model = load_model(path)
        print(f'✅ Loaded model for {self.ticker}')

        with open('Results/results.json', 'r') as f:
            best_params = json.load(f)
        self.hyperparams = best_params[self.ticker]['Hyperparameters']
        print(f'✅ Loaded hyperparameters for {self.ticker}')

        with open(f'Scalers/{base_ticker}_feature_scaler.pkl', 'rb') as f:
            self.feature_scaler = joblib.load(f)
        with open(f'Scalers/{base_ticker}_target_scaler.pkl', 'rb') as f:
            self.target_scaler = joblib.load(f)
        print(f'✅ Loaded feature & target scalers for {self.ticker}')

    # 2 ----> Fetches + Creates Dataset
    def fetch_data(self):
        seq_len = self.hyperparams['sequence_length']
        self.dataset = yf.Ticker(self.ticker).history(period=f'{seq_len+20}d', interval='1d').dropna()
        print(f'✅ Fetched latest data for {self.ticker}')

        features = self.dataset[['Open', 'High', 'Low', 'Volume']]
        target = self.dataset[['Close']]

        self.scaled_features = self.feature_scaler.transform(features)
        self.scaled_target = self.target_scaler.transform(target)

    # 3 ----> Predict Next-Day Close
    def make_prediction(self):
        if self.model is None or self.scaled_features is None or self.target_scaler is None:
            raise ValueError("Model, scaled features, or target scaler not loaded.")

        seq_len = self.hyperparams['sequence_length']
        if len(self.scaled_features) < seq_len:
            raise ValueError(f"Need at least {seq_len} days of data.")

        X_input = self.scaled_features[-seq_len:]
        X_input = np.reshape(X_input, (1, seq_len, X_input.shape[1]))

        pred_scaled = self.model.predict(X_input, verbose=0)
        predicted_close = self.target_scaler.inverse_transform(pred_scaled)[0][0]

        last_date = self.dataset.index[-1]
        next_date = last_date + BDay(1)

        recent_closes = self.dataset['Close'].tail(5).values
        volatility = np.std(recent_closes) / np.mean(recent_closes) if len(recent_closes) >= 2 else 0
        confidence = round(min(max(0, 1 - volatility * 10) * 100, 99.9), 2)

        currency = "₹" if '.NS' in self.ticker else "$"
        print(f"\n📅 Last Available Date: {last_date.date()}")
        print(f"📅 Predicted Date: {next_date.date()}")
        print(f"💹 Predicted Next-Day Close for {self.ticker}: {currency}{predicted_close:.2f}")
        print(f"📈 Confidence Score: {confidence:.2f}%")

        return predicted_close

    # 4 ----> Recursive Multi-Day Prediction
    def make_recursive_predictions(self, n_days=4):
        """Predicts the next n_days recursively using the trained LSTM model."""
        if self.model is None or self.scaled_features is None:
            raise ValueError("Model and scaled features must be loaded first.")

        seq_len = self.hyperparams['sequence_length']
        data = self.scaled_features.copy()  # to avoid modifying original data

        last_date = self.dataset.index[-1]
        self.future_predictions = []

        for i in range(n_days):
            X_input = data[-seq_len:]
            X_input = np.reshape(X_input, (1, seq_len, X_input.shape[1]))

            pred_scaled = self.model.predict(X_input, verbose=0)
            predicted_close = self.target_scaler.inverse_transform(pred_scaled)[0][0]

            next_date = last_date + BDay(i + 1)
            currency = "₹" if '.NS' in self.ticker else "$"
            self.future_predictions.append({
                "Date": next_date.date(),
                "Predicted_Close": float(predicted_close)
            })

            # ---- UPDATE INPUT SEQUENCE ----
            # Create a new row: use last known feature row but replace Close with predicted value
            last_row_features = self.dataset[['Open', 'High', 'Low', 'Volume']].iloc[-1].values
            new_feature_row = np.array(last_row_features).reshape(1, -1)
            new_feature_scaled = self.feature_scaler.transform(new_feature_row)
            data = np.vstack([data, new_feature_scaled])  # add new prediction

        # Confidence based on recent volatility (last 5 days)
        recent_closes = self.dataset['Close'].tail(5).values
        volatility = np.std(recent_closes) / np.mean(recent_closes) if len(recent_closes) >= 2 else 0
        confidence = round(min(max(0, 1 - volatility * 10) * 100, 99.9), 2)

        # ---- PRINT RESULTS ----
        print("\n🔁 Recursive Multi-Day Forecast:")
        print(f"📈 Confidence Score for next day : {confidence:.2f}%")
        for pred in self.future_predictions:
            print(f"📅 {pred['Date']} → 💹 Predicted Close: {currency}{pred['Predicted_Close']:.2f}")

        return self.future_predictions

    # 4 ----> Return close prices
    def get_close_prices(self):
        Close_prices = []
        dates = []
        for dicti in self.future_predictions:
          Close_prices.append(dicti['Predicted_Close'])
          dates.append(dicti['Date'])
        return dates, Close_prices


    # 5 ----> Runs Full Pipeline
    def run_pipeline(self, recursive_days=0):
        self.load_all()
        self.fetch_data()

        if recursive_days > 0:
            return self.make_recursive_predictions(n_days=recursive_days)
        else:
            return self.make_prediction()

In [40]:
obj = TestModel2('TCS.NS')
obj.run_pipeline(recursive_days=5) # use recursive days upto 5 days
dates, prices = obj.get_close_prices()

✅ Loaded model for TCS.NS
✅ Loaded hyperparameters for TCS.NS
✅ Loaded feature & target scalers for TCS.NS
✅ Fetched latest data for TCS.NS


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(



🔁 Recursive Multi-Day Forecast:
📈 Confidence Score for next day : 94.86%
📅 2025-11-03 → 💹 Predicted Close: ₹3109.96
📅 2025-11-04 → 💹 Predicted Close: ₹3110.13
📅 2025-11-05 → 💹 Predicted Close: ₹3110.77
📅 2025-11-06 → 💹 Predicted Close: ₹3111.50
📅 2025-11-07 → 💹 Predicted Close: ₹3110.65
